# Arboles de Decisión


In [1]:
# Esto agrega al python path el directorio ..
import sys
sys.path.append("..")

import numpy as np
import config
from data_builder import load_test_data, load_dev_data, load_small_dev_data


df, target = load_small_dev_data()


# Optimización de hiperparámetros

Busquemos los mejores (posibles) hiperparámetros

Para eso, primero veamos qué hiperparámetros nos provee la implementación de SKLearn.

In [2]:
from transformers import extractor



extractor.fit(df)
X = extractor.transform(df)



(8099, 376)


Algunas observaciones previas:

El árbol que se genera no es usando ID3 sino CART. Una pequeña introducción
se puede ver en

http://scikit-learn.org/stable/modules/tree.html#tree-algorithms-id3-c4-5-c5-0-and-cart

No está muy completo el asunto ahí, habría que hurgar algún paper extra

Opciones:

- criterion: gini o entropy. Son dos medidas distintas
- splitter: No estoy muy seguro
- max_depth: máxima profundidad del árbol
- max_features: qué porcentaje de variables tomo a la hora de partir un nodo.
  Estas variables se eligen aleatoriamente
- min_samples_split: cuántos elementos tengo que tener en un nodo para decidir
  partirlo

In [6]:
from search import find_best_classifier
from sklearn.tree import DecisionTreeClassifier

options = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': range(2, 52, 8),
    'max_features': np.arange(0.1, 1.0, 0.2),
    'min_samples_split': range(2, 102, 10),
}

X = df
y = target
classifier = find_best_classifier(
    X, y, DecisionTreeClassifier, param_distributions=options, 
    scoring='roc_auc', cv=10, n_jobs=6, n_iter=10)    

Buscando parámetros para DecisionTreeClassifier


UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 863: ordinal not in range(128)

In [ ]:
from helpers import get_scores

get_scores(classifier)